# **Welcome to the Jupyter notebook showing programmatically discovery and access of WHOS data**

This notebook is used to programmatically access WHOS DAB functionalities, in this case through the OGC OM-JSON based API, documented and available for tests also here: https://whos.geodab.eu/gs-service/om-api

The WHOS portal can also be used to help debug your python notebook: https://whos.geodab.eu/gs-service/whos/search.html

## **Step 1: Install dab-py**

The Python client library `dab-py` is available on PyPI and GitHub: https://github.com/ESSI-Lab/dab-py

In [ ]:
!pip install --upgrade dab-py

## **Step 2: Main Code Tutorial**

### **2.1: Import Library**
Import the main `dabpy` library to access WHOS API `om_api` functionalities.

In [ ]:
from dabpy import WHOSClient, Constraints
from IPython.display import display

### **2.2: Initialize WHOS Client (Token & View)**
Replace with your WHOS API token and optional view. This initializes the client to interact with the WHOS API.

As a prerequisite to obtain programmatically access to WHOS, a token is required. It can be obtained after registration at https://whos.geodab.eu/gs-service/whos/registration.html.

In [ ]:
# Replace with your WHOS API token and optional view
token = "my-token"  # replace with your actual token
view = "whos"
client = WHOSClient(token=token, view=view)

### **2.3: Define Feature Constraints**
Feature retrieval in the WHOS API can be restricted using optional constraints.

#### 2.3.1: Default: Spatial Constraints *(bounding box)*  
Set the geographic bounding box for spatial queries. The example uses a region in Finland.

In [ ]:
# Define bounding box coordinates (south, west, north, east)
south = 60.398
west = 22.149
north = 60.690
east = 22.730

In this example, only spatial constraints (bounding box) are applied, while the other filters remain optional.

In [ ]:
# Create feature constraints
constraints = Constraints(bbox = (south, west, north, east))

#### 2.3.2: Optional: Other Constraints
Additional optional constraints can be applied to further filter the results: 
- **Observed property** : filter features based on the type of measurement or variable, such as water level, precipitation, discharge, etc. 
- **Ontology** : specify which ontology to use to expand the observed property search term or URI with synonyms and related narrower concepts. Two ontologies are available: `whos` and `his-central`.
- **Country**: using Country code (ISO3).
- **Provider**: data provider identifier.
- The other constraints are listed from the documented API https://whos.geodab.eu/gs-service/om-api/

If optional constraints are specified, update the code with detail constraints:

In [ ]:
# You can also specify additional constraints as needed, for example:
'''
constraints = Constraints(
    bbox = (south, west, north, east),
    observedProperty = "example_property",
    ontology = "example_ontology",
    country = "example_country",
    provider = "example_provider",
    spatialRelation = "example_spatial_relation",
    feature = "example_feature_id",  #can also use features[index_number].id directly
    localFeatureIdentifier = "example_local_id",
    observationIdentifier = "example_obs_id",
    beginPosition = "2025-01-01T00:00:00Z",
    endPosition = "2025-02-01T00:00:00Z",
    predefinedLayer = "example_layer",
    timeInterpolation = "example_interpolation",
    intendedObservationSpacing = "PT1H",  # ISO 8601 duration, e.g., 1 hour
    aggregationDuration = "P1D"  # ISO 8601 duration, e.g., 1 day
)
'''

### **2.4: Get `Features`**

#### 2.4.1: Retrieve features as Python objects
Fetch features from the WHOS API URL as Python `Feature` objects using the previously defined constraints.

In [ ]:
# 01.1.1: Get Features as Python objects
features = client.get_features(constraints)

The retrieved **features** are stored in a `FeaturesCollection`.  
Then convert the current page to a Pandas DataFrame for easier inspection and analysis.then converted into a Pandas DataFrame for easier inspection, filtering, and analysis.

In [ ]:
# 01.1.2: (optional: Convert Features to DataFrame if needed)
features_df = features.to_df()
display(features_df)

#### 2.4.2: (If needed / more data exists) Fetch the next page
If the dataset is **not completed** (contains more than 10 datas) - like the above data, fetch the next page using `.next()`

In [ ]:
# 01.2.1: Fetch next page (if available).
features.next()

# 01.2.2: (optional) Convert current page features to DataFrame.
features_df = features.to_df()
display(features_df)

### **2.5: Get `Observations`**

#### 2.5.1: Retrieve observations as Python objects (using previous constraints)

In this part, the constraints are automatically inherited from Step 2.3, the same as those used for `GET Features`.

If the dataset is not completed (contains more than 10 datas), fetch the next page using `.next()` *(see Step 2.4.2)*.

In [ ]:
# 02.1.1: Retrieve observations matching the previously defined constraints (only bbox).
observations = client.get_observations(constraints)

The retrieved **observations** are then converted into a Pandas DataFrame for easier inspection, filtering, and analysis.

In [ ]:
# 02.1.2: (optional: Convert Observations to DataFrame if needed)
observations_df = client.observations_to_df(observations)
display(observations_df)

#### 2.5.2: Retrieve observations as Python objects (using new constraints)

Define a new set of constraints (`new_constraints`) to retrieve observations.  
In this example, retrieve observations for a specific feature (station).
There are two ways to specify the feature ID:
  1. Directly provide the ID as a string, e.g.:  
     ```python
     feature="9413350CE1DF751D17223F864EC4C9FA33BBC64E"
     ```
  2. Extract the ID from a previously retrieved features list, e.g.:   
     ```python
     feature=features[9].id
     ```
     - `features[9]` → selects the 10th feature (station) in the list (index 9)  
     - `.id` → accesses its unique identifier, required by the API

In [ ]:
# 02.2.1: (or) retrieve observations from a different constraints - by defining new_constraints.
new_constraints = Constraints(feature=features[9].id)
observations_new_constraints = client.get_observations(new_constraints)

The retrieved **observations** are then converted into a Pandas DataFrame for easier inspection, filtering, and analysis.

In [ ]:
# 02.2.2: (optional: Convert Observations to DataFrame if needed)
observations_new_constraints_df = client.observations_to_df(observations_new_constraints)
display(observations_new_constraints_df)

### **2.6: Get the Data Points**
#### 2.6.1: Retrieve Observation With Data Points

Retrieve the full time-series data for a selected observation within a specified time range. It shows the actual code that retrieves the data, to separate the action (API call) from what to do next with the data.

In [ ]:
# 03.1: Get first observation with data points
obs_with_data = client.get_observation_with_data(observations_new_constraints[0].id, begin="2025-01-01T00:00:00Z", end="2025-02-01T00:00:00Z")

#### 2.6.2: (optional) Convert to DataFrame for analysis

The returned data can be plotted or further analyzed as needed.

In [ ]:
# 03.2: (optional: Convert Observation Points to DataFrame if needed)
obs_points_df = client.points_to_df(obs_with_data)
display(obs_points_df)

#### 2.6.3: (Optional) Example of Graphical Time-Series
This step demonstrates visualization, which is a different action from simply retrieving or tabulating data.  

In [ ]:
# 03.3: (optional: Example of Graphical Time-Series)
client.plot_observation(obs_with_data, "Example of Time-series, custom your own title")

## **Step 3: Full Code**

This part contains the complete working example from steps 2.1–2.6.

It can be run as a single block to reproduce all results.

```python
from dabpy import WHOSClient, Constraints
from IPython.display import display

# Replace with your WHOS API token and optional view
token = "my-token"  # replace with your actual token
view = "whos"
client = WHOSClient(token=token, view=view)


## 00 DEFINE FEATURE CONSTRAINTS
# Define bounding box coordinates (south, west, north, east), example of Finland.
south = 60.398
west = 22.149
north = 60.690
east = 22.730
# Create feature constraints, only spatial constraints are applied, while the other filters remain optional.
constraints = Constraints(bbox = (south, west, north, east))


## 01 GET FEATURES
# 01.1: Retrieve features matching the previously defined constraints (only bbox).
features = client.get_features(constraints)
# Use 'paginate=True' - features = client.get_features(constraints, paginate=True) to fetch all pages.

# 01.2: (optional: Convert Features to DataFrame if needed).
features_df = client.features_to_df(features)
display(features_df)


## 02 GET OBSERVATIONS
# 02.1.1: Retrieve observations matching the previously defined constraints (only bbox).
observations = client.get_observations(constraints)
# Use 'paginate=True' - observations = client.get_observations(constraints, paginate=True) to fetch all pages.

# 02.1.2: (optional: Convert Observations to DataFrame if needed).
observations_df = client.observations_to_df(observations)
display(observations_df)

# 02.2.1: (or retrieve observations from a different constraints - by defining new_constraints).
new_constraints = Constraints(feature=features[9].id)
observations_new_constraints = client.get_observations(new_constraints)
# Use 'paginate=True' - observations_new_constraints = client.get_observations(new_constraints, paginate=True) to fetch all pages.

# 02.2.2: (optional: Convert Observations to DataFrame if needed)
observations_new_constraints_df = client.observations_to_df(observations_new_constraints)
display(observations_new_constraints_df)


## 03 GET DATA POINTS
# 03.1: Get first observation with data points
obs_with_data = client.get_observation_with_data(observations_new_constraints[0].id, begin="2025-01-01T00:00:00Z", end="2025-02-01T00:00:00Z")
# 03.2: (optional: Convert Observation Points to DataFrame if needed)
obs_points_df = client.points_to_df(obs_with_data)
display(obs_points_df)
# 03.3: (optional: Example of Graphical Time-Series)
client.plot_observation(obs_with_data, "Example of Time-series, custom your own title")